### importing dependencies

In [2]:
import pandas as pd

import numpy as np

import difflib   ## if user enter any spelling mistake in search then difflib will be used to find the close matching string for it from dataset
#by using function  'difflib.get_close_matches() '

from sklearn.feature_extraction.text import TfidfVectorizer ## it will convert text data into numeric value(feature vectors)..
##and TfidfVectorizer also reduces weight of common words which appers frequently in many documents like ('is','the')

from sklearn.metrics.pairwise import cosine_similarity # it will measure how two texts are similar  &
##it is used when length of document or size of vector is not depend on the similarity score &  find which movie has highest similarity score

### data collection and preprocessing

In [4]:
##loding data from csv file to pandas dataframe
movies_data = pd.read_csv("movies.csv")
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
# number of row ansd column in dataframe
movies_data.shape

(4803, 24)

In [8]:
#selecting relevant features for recommendation
selected_features = ['genres','keywords', 'tagline', 'cast', 'director']
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [9]:
## checking if null values are there 
movies_data.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   4
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       1
director                  30
dtype: int64

In [10]:
# replacing null values with null string
for features in selected_features:
    movies_data[features] = movies_data[features].fillna('')

In [12]:
#combining all the 5 selected features
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' ' +movies_data['cast']+' '+movies_data['director']
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [13]:
#converting text data to feature vectors(our data is in text format and for finding cosine similarity we want to convert text data to numeric data
vectorizer = TfidfVectorizer()

In [15]:
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

### cosine similarity

In [17]:
## getting similarity score from cosine similarity
similarity = cosine_similarity(feature_vectors)
similarity

array([[1.        , 0.07219487, 0.037733  , ..., 0.        , 0.        ,
        0.        ],
       [0.07219487, 1.        , 0.03281499, ..., 0.03575545, 0.        ,
        0.        ],
       [0.037733  , 0.03281499, 1.        , ..., 0.        , 0.05389661,
        0.        ],
       ...,
       [0.        , 0.03575545, 0.        , ..., 1.        , 0.        ,
        0.02651502],
       [0.        , 0.        , 0.05389661, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02651502, 0.        ,
        1.        ]])

In [18]:
similarity.shape  ##(here (x, y) where x = index of movie & y = array of similarity score with all other movies

(4803, 4803)

In [19]:
##gettting movie name from user
movie_name = input("enter your favourite movie name:")

enter your favourite movie name: avtar


In [21]:
#creating list with all the movie name given in the dataset

list_of_all_titles = movies_data['title'].tolist()
list_of_all_titles

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [23]:
#finding closest match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
find_close_match

['Avatar', 'Salvador', 'Water']

In [24]:
close_match = find_close_match[0]
close_match

'Avatar'

In [25]:
#find the index of the movie with title

index_of_movie = movies_data[movies_data.title == close_match]['index'].values[0]
index_of_movie

0

In [31]:
## getting list of similar movies

similarity_score = list(enumerate(similarity[index_of_movie]))  # enumerate used to just carry out run a loop
similarity_score

[(0, 1.0000000000000002),
 (1, 0.07219486822992491),
 (2, 0.03773299957717928),
 (3, 0.012520204623868908),
 (4, 0.10702574467235304),
 (5, 0.07786899789424222),
 (6, 0.008237143013608844),
 (7, 0.03613473061484884),
 (8, 0.02960930964063025),
 (9, 0.02628716743995174),
 (10, 0.09261074046755373),
 (11, 0.012717759249124133),
 (12, 0.027217360083100117),
 (13, 0.02956975252334751),
 (14, 0.06915925473724742),
 (15, 0.01955159449930903),
 (16, 0.03426340578061641),
 (17, 0.026036564614294145),
 (18, 0.057147592666724124),
 (19, 0.0389505953521203),
 (20, 0.03971480215415495),
 (21, 0.01201480380565613),
 (22, 0.03043869426198959),
 (23, 0.045926535588179496),
 (24, 0.04623989017965259),
 (25, 0.042849260959502256),
 (26, 0.07010711150614286),
 (27, 0.037198076232328854),
 (28, 0.04083909796927843),
 (29, 0.03858648330156397),
 (30, 0.07893753610792024),
 (31, 0.060555221380551486),
 (32, 0.030362745635800832),
 (33, 0.03591021470068869),
 (34, 0.0),
 (35, 0.03769674103474844),
 (36, 0.0

In [28]:
len(similarity_score)

4803

In [30]:
#sorting the movies based on there similarity score

sorted_similar_movies = sorted(similarity_score, key=lambda x:x[1],reverse=True)       #key here x represnts similarity_score list.
##so sorting is done by using key value where x[1] is  1.0000000000000002 these 2nd column values

sorted_similar_movies

[(0, 1.0000000000000002),
 (3158, 0.24946766307532411),
 (2403, 0.24841462595906275),
 (94, 0.24505931974059814),
 (56, 0.2037806964828543),
 (47, 0.20115287461144912),
 (1053, 0.19702752258651426),
 (838, 0.18017023369312357),
 (3730, 0.17646241185313413),
 (4593, 0.1744884579741517),
 (239, 0.17441748680810654),
 (1531, 0.1682605817219649),
 (2696, 0.16503460259176522),
 (812, 0.1606230190749179),
 (643, 0.15644455512484967),
 (4401, 0.15468923545220403),
 (2198, 0.15217161971893786),
 (770, 0.15025726727753488),
 (1951, 0.14933372705282924),
 (2229, 0.14661801285492257),
 (1922, 0.1448197430191312),
 (206, 0.14226144606175545),
 (3208, 0.1401230206493547),
 (1759, 0.13899056016968867),
 (43, 0.1348209130228474),
 (1473, 0.13476547670086914),
 (278, 0.13291021545504),
 (158, 0.13252892131627672),
 (1650, 0.13024318650645417),
 (1275, 0.12602216304791153),
 (3439, 0.12480340331169382),
 (661, 0.12153002734138185),
 (3202, 0.12144749322246053),
 (4332, 0.12002556168548507),
 (3105, 0.1

In [33]:
#print the name of similar movies based on the index

print('Movies Suggested For You!!')

i = 1
for movie in sorted_similar_movies:
    index = movie[0]  ##here index will store first column value in sorted_similar_movies
    title_from_index = movies_data[movies_data.index==index]['title'].values[0]
    if i<31:
        print(i,',',title_from_index)
        i+=1

Movies Suggested For You!!
1 , Avatar
2 , Alien
3 , Aliens
4 , Guardians of the Galaxy
5 , Star Trek Beyond
6 , Star Trek Into Darkness
7 , Galaxy Quest
8 , Alien³
9 , Cargo
10 , Trekkies
11 , Gravity
12 , Moonraker
13 , Jason X
14 , Pocahontas
15 , Space Cowboys
16 , The Helix... Loaded
17 , Lockout
18 , Event Horizon
19 , Space Dogs
20 , Machete Kills
21 , Gettysburg
22 , Clash of the Titans
23 , Star Wars: Clone Wars: Volume 1
24 , The Right Stuff
25 , Terminator Salvation
26 , The Astronaut's Wife
27 , Planet of the Apes
28 , Star Trek
29 , Wing Commander
30 , Sunshine


# movie recommendation system

In [39]:

movie_name = input("enter your favourite movie name:")

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_movie])) 

sorted_similar_movies = sorted(similarity_score, key=lambda x:x[1],reverse=True) 

print('Movies Suggested For You!!')

i = 1
for movie in sorted_similar_movies:
    index = movie[0]  ##here index will store first column value in sorted_similar_movies
    title_from_index = movies_data[movies_data.index==index]['title'].values[0]
    if i<31:
        print(i,',',title_from_index)
        i+=1

enter your favourite movie name: big fish


Movies Suggested For You!!
1 , Big Fish
2 , Dark Shadows
3 , Alice in Wonderland
4 , Planet of the Apes
5 , Corpse Bride
6 , Muppets from Space
7 , The Heart of Me
8 , Zoom
9 , The Young and Prodigious T.S. Spivet
10 , Rudderless
11 , Finding Nemo
12 , Begin Again
13 , Cinderella
14 , Down Terrace
15 , A Room with a View
16 , Howards End
17 , Alice Through the Looking Glass
18 , The Curse of the Were-Rabbit
19 , Country Strong
20 , Charlie and the Chocolate Factory
21 , Waitress
22 , The Revenant
23 , Fish Tank
24 , The Princess Bride
25 , The Lone Ranger
26 , Red River
27 , Martian Child
28 , Breakfast of Champions
29 , Solitary Man
30 , Duel in the Sun
